### 1. 介绍

- **LE**：拉普拉斯特征映射（Laplacian Eigenmaps），一种非线性降维算法，用于对高维数据保持局部性进行降维。

- **注意**: 权重矩阵$W_{i,j}$是根据高维数据点$x_i$和$x_j$之间的相似性来定义的，而降维后的数据$y_i$和$y_j$之间的距离也是用的这个$W_{i,j}$，因此可以令高维空间中相似的数据点在低维空间中的距离也相似，即在降维中保持局部性。

- **原始目标函数**：
  $$\min_{Y} \sum_{i,j}  \|y_i - y_j\|^2 W_{ij}$$
  $$s.t. \quad Y^T D Y = I$$
  其中，$Y$为降维后的数据，$W$为邻接矩阵，$D$为度矩阵。

  - **符号描述**：
    - $ Y $：表示低维空间中的数据点。
    - $ \|y_i - y_j\| $：表示低维空间中数据点 $ y_i $ 和 $ y_j $ 之间的欧氏距离。
    - $ W_{ij} $：表示高维空间中数据点 $ x_i $ 和 $ x_j $ 之间的权重，通常反映了数据点之间的相似性。权重 $ W_{ij} $ 越大，表示 $ x_i $ 和 $ x_j $ 越相似。
    - $ D $：是一个对角矩阵，其对角元素 $ D_{ii} $ 是 $ W $ 的行和，即 $ D_{ii} = \sum_j W_{ij} $。
    - $ I $：是单位矩阵。

  - **目标函数的几何意义**：保持数据的局部相似性（基于欧氏距离），使得降维后的点之间的距离尽可能相似。通过最小化低维空间中相邻数据点的距离平方和来保持高维空间中数据点的局部几何结构，约束条件则确保了低维嵌入的标准化和稳定性，具体有：
  
    1. **保持局部结构**：目标函数中的和 $ \sum_{i,j} \|y_i - y_j\|^2 W_{ij} $ 旨在最小化低维空间中相邻数据点之间的距  离。这意味着，如果高维空间中两个数据点 $ x_i $ 和 $ x_j $ 的权重 $ W_{ij} $ 较大（即它们在高维空间中很相似），那么在低维空间中对应的点 $ y_i $ 和 $ y_j $ 之间的距离也应该很小。因此，这个目标函数能够保持数据点的局部几何结构。
    2. **局部性约束**：约束条件 $ Y^T D Y = I $ 确保了低维嵌入的标准化。这个约束实际上是为了避免低维嵌入的退化（例如所有点都映射到原点），并保证低维空间中的点不会因为缩放或旋转而影响嵌入结果。 


- **流程**：
  1. **构建邻接矩阵**：按照$\epsilon$-邻近, 对每个样本选取最近的$K$个点，构建邻接矩阵$W$，若不在范围内则权重为0。
    $$W_{ij} = \begin{cases} \exp(-\frac{\|x_i - x_j\|^2}{2\sigma^2}) & \text{if } \|x_i - x_j\| < \epsilon
     \\ 0 & \text{, else} \end{cases}$$
  2. 计算度矩阵
    $$D_{ii} = \sum_j W_{ij}$$
  3. 计算拉普拉斯矩阵
    $$L = D - W$$
  4. 计算特征值和特征向量, 计算出降维后的数据$Y$
    $$LY = \Lambda DY$$

- **优点**：

    - 计算效率高
    - 具有保持局部性的特点
    - 与聚类有自然的联系。

### 2. 过程证明

#### 2.1. 注意事项：

注意以下两点：

- $$\sum_{i,j} y_i W_{i,j} y_j = \bf{W} $$
- $$\sum_{i} y_i W_{i,j} y_i = \bf{D} $$



#### 2.2. 流程推导：

这个问题涉及到从优化目标函数到广义特征值问题的推导。我们从以下目标函数和约束条件开始：
$$
\min_{Y} \sum_{i,j} \|y_i - y_j\|^2 W_{ij}
$$
$$
s.t. \quad Y^T D Y = I
$$

首先，目标函数中的平方距离可以展开为：
$$
\sum_{i,j} \|y_i - y_j\|^2 W_{ij} = \sum_{i,j} (\|y_i\|^2 + \|y_j\|^2 - 2y_i^T y_j) W_{ij}
$$
由于 $\|y_i\|^2 W_{ij}$ 和 $\|y_j\|^2 W_{ij}$ 在求和时对称，可以合并为：
$$
= \sum_{i,j} \left(\|y_i\|^2 W_{ij} + \|y_j\|^2 W_{ij} - 2y_i^T y_j W_{ij}\right)
$$
$$
= 2 \sum_{i} \|y_i\|^2 D_{ii} - 2 \sum_{i,j} y_i^T y_j W_{ij}
$$
其中， $D_{ii} = \sum_{j} W_{ij}$ 是节点 $i$ 的度。

目标函数可以写成矩阵形式：
$$
\sum_{i,j} \|y_i - y_j\|^2 W_{ij} = 2 \left( \text{Tr}(Y^T D Y) - \text{Tr}(Y^T W Y) \right)
$$
其中， $\text{Tr}(\cdot)$ 表示矩阵的迹（trace）。
因此，目标函数变为：
$$
\min_{Y} 2 \left( \text{Tr}(Y^T D Y) - \text{Tr}(Y^T W Y) \right)
$$
由于常数因子2不影响最优化结果，可以简化为：
$$
\min_{Y} \left( \text{Tr}(Y^T D Y) - \text{Tr}(Y^T W Y) \right)
$$
这等同于最大化 $\text{Tr}(Y^T W Y)$，因为 $\text{Tr}(Y^T D Y)$ 是常数。

引入图拉普拉斯矩阵 $L = D - W$，目标函数变为：
$$
\min_{Y} \text{Tr}(Y^T L Y)
$$

考虑约束条件 $Y^T D Y = I$，这个约束条件在优化问题中用于避免退化解，用拉格朗日乘子法引入拉格朗日乘子 $\Lambda$，得到拉格朗日函数：

$$ L(Y, \Lambda) = \text{Tr}(Y^T L Y) - \text{Tr}(\Lambda (Y^T D Y - I)) $$

求导并令导数为0，得到最优解：
$$
LY = \Lambda DY
$$
这是一个典型的广义特征值问题，其中 $\Lambda$ 是特征值，$Y$ 是对应的特征向量，同时也是对应降维后的数据。